In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


print("Fetching live market data... Please wait.\n")


target_ticker = "JSWSTEEL.NS"
peers = ["TATASTEEL.NS", "JINDALSTEL.NS", "SAIL.NS"]


jsw = yf.Ticker(target_ticker)
current_price = jsw.info.get('currentPrice', 0)
shares_outstanding = jsw.info.get('sharesOutstanding', 0) / 10**7  # Convert to Crores
beta = jsw.info.get('beta', 1.15)  # Fallback to 1.15 if missing


peer_data = []
for p in peers:
    ticker = yf.Ticker(p)
    info = ticker.info

    
    forward_pe = info.get('forwardPE', 0)
    ev_ebitda = info.get('enterpriseToEbitda', 0)
    peer_data.append({'Ticker': p, 'EV/EBITDA': ev_ebitda, 'P/E': forward_pe})

peer_df = pd.DataFrame(peer_data)
avg_ev_ebitda = peer_df['EV/EBITDA'].mean()
avg_pe = peer_df['P/E'].mean()

print(f"JSW Steel Current Price: ₹{current_price}")
print(f"JSW Beta: {beta}")
print(f"Peer Group Avg EV/EBITDA: {avg_ev_ebitda:.2f}x")
print(f"Peer Group Avg P/E: {avg_pe:.2f}x\n")




risk_free_rate = 0.0665  # 6.65%
market_return = 0.1345   
cost_of_debt_pre_tax = 0.095
tax_rate = 0.252

# WACC Calculation Function
def calculate_wacc(beta, rf, rm, kd, tax, equity_weight=0.60):
    ke = rf + beta * (rm - rf)
    kd_post_tax = kd * (1 - tax)
    wacc = (equity_weight * ke) + ((1 - equity_weight) * kd_post_tax)
    return wacc

wacc = calculate_wacc(beta, risk_free_rate, market_return, cost_of_debt_pre_tax, tax_rate)
print(f"Calculated WACC: {wacc:.2%}")

# DCF Calculation
# Projected FCFF for FY26-FY30 (From your Excel Table)
# [FY26, FY27, FY28, FY29, FY30]
fcff_projections = [6746, 12552, 15909, 19205, 22450] # In Crores
terminal_growth = 0.04

# NPV Calculation
npv_explicit = np.npv(wacc, fcff_projections) # Numpy's NPV function

# Terminal Value
terminal_value = (fcff_projections[-1] * (1 + terminal_growth)) / (wacc - terminal_growth)
pv_terminal_value = terminal_value / ((1 + wacc) ** 5)

# Enterprise Value & Equity Value
enterprise_value = npv_explicit + pv_terminal_value
net_debt = 65500 # FY25 Actual
equity_value = enterprise_value - net_debt
intrinsic_share_price = equity_value / shares_outstanding

print(f"Intrinsic DCF Share Price: ₹{intrinsic_share_price:.2f}")

# ==========================================
# 3. RELATIVE VALUATION (LOGIC)
# ==========================================

# Using Forward Estimates (FY27E from Excel)
projected_ebitda_fy27 = 37761 
projected_pat_fy27 = 15425

# Implied Price via EV/EBITDA
implied_ev_relative = projected_ebitda_fy27 * avg_ev_ebitda
implied_eq_relative = implied_ev_relative - net_debt
price_relative_ev = implied_eq_relative / shares_outstanding

# Implied Price via P/E
price_relative_pe = (projected_pat_fy27 * avg_pe) / shares_outstanding

print(f"Implied Price (EV/EBITDA): ₹{price_relative_ev:.2f}")
print(f"Implied Price (P/E): ₹{price_relative_pe:.2f}\n")

# ==========================================
# 4. FOOTBALL FIELD CHART (VISUALIZATION)
# ==========================================

labels = ['52W Low-High', 'Relative (P/E)', 'Relative (EV/EBITDA)', 'DCF (Intrinsic)']
# Min, Max values for the bars
values = [
    (jsw.info.get('fiftyTwoWeekLow'), jsw.info.get('fiftyTwoWeekHigh')),
    (price_relative_pe * 0.9, price_relative_pe * 1.1), # +/- 10% range
    (price_relative_ev * 0.9, price_relative_ev * 1.1),
    (intrinsic_share_price * 0.9, intrinsic_share_price * 1.1)
]

fig, ax = plt.subplots(figsize=(10, 6))

for i, (min_val, max_val) in enumerate(values):
    mid_val = (min_val + max_val) / 2
    ax.plot([min_val, max_val], [i, i], color='skyblue', linewidth=10, solid_capstyle='round')
    ax.scatter(mid_val, i, color='navy', s=100, zorder=3) # Dot for base case
    ax.text(max_val + 20, i, f"₹{int(mid_val)}", va='center', fontweight='bold')

ax.set_yticks(range(len(labels)))
ax.set_yticklabels(labels)
ax.set_xlabel('Share Price (₹)')
ax.set_title(f'JSW Steel: Valuation Football Field (As of {pd.Timestamp.now().date()})')
ax.grid(axis='x', linestyle='--', alpha=0.7)

# Save or Show
print("Generating Valuation Football Field Dashboard...")
plt.tight_layout()
plt.show()

Fetching live market data... Please wait.



Timeout: Failed to perform, curl: (28) Connection timed out after 30016 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.